# PyLightcurve core culculations

The core culculations are provided to offer maximum efficiency and help you 
**build you own scripts or codes** upon the functionalities of PyLigthcurve.

The functions that are included in the core calulations are:

- **```plc.exotethys```**: calculates the limb-darkening coefficients
- **```plc.fp_over_fs```**: calculates the planet-to-star flux ratio (flux emmitted + reflected by the planet)
- **```plc.convert_to_bjd_tdb```**: converts a time array from different time formats to BJD<sub>TDB</sub>
- **```plc.convert_to_jd_utc```**: converts a time array from different time formats to JD<sub>UTC</sub>
- **```plc.convert_to_relflux```**: converts a flux array from magnitude to relative flux
- **```plc.planet_orbit```**: calculates the (x,y,z) position of the planet in a cartesian frame with the star at (0,0,0), the Earth at (+infinity,0,0) and the periastron located on the −y axis for $\omega$=0.
- **```plc.planet_star_projected_distance```**: calculates the distance between the planet and the star relatively to the stellar radius, as projected on the celestial sphere.
- **```plc.planet_phase```**: calculates the planet phase with respect to the transit mid-time (time of conjunction when the planet is behind the star, no units)
- **```plc.transit_depth```**: calculates the transit depth (in relative flux)
- **```plc.transit_duration```**: calculates the transit duration (in days)
- **```plc.transit_t12```**: calculates the time between transit contact points 1 and 2 (in days)
- **```plc.eclipse_mid_time```**: calculates the eclipse mid-time (time of conjunction when the planet is behind the star, in days BJD<sub>TDB</sub>)
- **```plc.eclipse_depth```**: calculates the eclipse depth (in relative flux)
- **```plc.eclipse_duration```**: calculates the eclipse duration (in days)
- **```plc.transit```**: calculates the transit model (in relative flux)
- **```plc.eclipse```**: calculates the eclipse model (in relative flux)
- **```plc.transit_integrated```**: calculates the exposure-integtared transit model (in relative flux)
- **```plc.eclipse_integrated```**: calculates the exposure-integtared eclipse model (in relative flux)
- Angle conversions 



In [1]:
%matplotlib notebook
import matplotlib.pyplot as plt
import pylightcurve as plc
import numpy as np

exoclock: Checking ephemerides database...
exoclock: Checking catalogues database...
exoclock: Checking ut database...
pylightcurve: Checking exotethys database...
pylightcurve: Checking photometry database...



## 1. Definitions

To use the core calculation in PyLightcurve we will need to provide some of the the following 
information.



### 1.1 Stellar and planetary parameters

Parameters that need to be provided by the user:

| variable name            | type       | decription                                |unit                    |
|:-------------------------|:----------:|:-----------------------------------------:|:----------------------:|
| ``ra``                   |``float``   |RA of the host star                        |degrees (ICRS)          |
| ``dec``                  |``float``   |DEC of the host star                       |degrees (ICRS)          |
| ``stellar_logg``         |``float``   |log(g) of the host star                    |log(cm/s<sup>2)         |
| ``stellar_temperature``  |``float``   |effecive temperature of the host star      |Kelvin                  |
| ``stellar_metallicity``  |``float``   |metallicity of the host star               |dex(Fe/H) or dex(M/H)   |
| ``rp_over_rs``           |``float``   |plant-to-star radius ratio                 |no units                |
| ``period``               |``float``   |orbital period                             |days                    |
| ``sma_over_rs``          |``float``   |orbital semi-major axis relatively <br> to the stellar radius       |no units|
| ``eccentricity``         |``float``   |orbital eccentricity                       |no units                |
| ``inclination``          |``float``   |orbital inclination                        |degrees                 |
| ``periastron``           |``float``   |orbital argument of periastron             |degrees                 |
| ``mid_time``             |``float``   |the time of conjunction with the planet in front of the star |days (BJD<sub>TDB</sub>)|
| ``albedo``               |``float``   |planet albedo                              |no units                |
| ``emissivity``           |``float``   |planet emissivity                          |no units                |


Parameters that can be provided by the user or calculated through PyLightcurve:

| variable name                  | type       | decription                                |unit                    |
|:-------------------------------|:----------:|:-----------------------------------------:|:----------------------:|
|``limb_darkening_coefficients``|``list``    |4-element list with the limb-darkening coefficients                        |no units          |
|``fp_over_fs``                 |``float``   |plant-to-star flux ratio (emmitted + reflected) |no units                |
| ``eclipse_mid_time``          |``float``   |the time of conjunction with the planet behind the star |days (BJD<sub>TDB</sub>)|


### 1.2 ```filter_name```

Some of the calculation in PyLighcurve are related to photometric/specroscopic filter. There are:
- the calculation of limb-darkening coefficients, and
- the calculation of the emmitted planetry flux.

The assosiated parameter is ```filter_name```.

There are different filters there are available by default in PyLighcurve, and you can see those by typing:

```python
plc.all_filters()
```

Alternatively, you can define your own filter as follows:
```python
plc.add_filter(filter_name, passband_file)
```

where the passband file should be a txt file containing two columns separated by space or tab:
- column 1: wavelength in Angstrom
- column 2: total throughput in electrons/photons.



### 1.3 ```wlrange = None```

For the cases where a photometric/specroscopic filter is needed, we can also specify a part of the filter's bandpass to be used, by defining the ```wlrange``` parameter (by default is ```None```). This is a list with two elements, indicating the limits (in Angstrom) within the filter bandpass that we want to use. The wavelength range needs to be within the limits of the photometric/specroscopic filter bandpass, otherwise the calsulation will fail (see Example 7 below).



### 1.4  ```method = 'claret'```

This parameter refers to the limb-darkening law that we want to use. The default value is ```claret```, and the other options are: ```linear``` and ```quad```.



### 1.5 ```stellar_model = 'Phoenix_2018'```

This parameter refers to stellar models that we want to use to calculate the limb-darkening coefficients
There is a range of stellar models availabe throught the exotethys pachage.
The default stellar model is the ```Phoenix_2018```. However some stellar parameters and some 
wavelengths, are not covered by all models. For example, ```Phoenix_2018``` are not availble 
for the ```irac4``` filter, or for very cool stars (e.g. 3000 K).
The other stellar models are: ```Atlas_2000```, ```Phoenix_2012_13```, ```Phoenix_drift_2012```, ```Stagger_2015```, ```Stagger_2018```.
Please consult the the [```ExoTETHyS```](https://github.com/ucl-exoplanets/ExoTETHyS) package page if an error occurs.
    
    
    
### 1.6  ```precision = 2```

This parameter is an option for the fulctions that calculate the relative flux during a transit or an eclipse. It refers to the precision of the numerical calculation of the Gauss–Legendre quadrature, the default value is 2, and the other options are 3, 4, ... 10, meaning that 20, 30, 40, ... 100 quadrature points will be used.

In [2]:
# For the examples below we will use the paraeters of the famous HD209458b:

ra = 330.795
dec = 18.884
stellar_logg = 4.36      
stellar_temperature = 6065.0
stellar_metallicity = 0.0
rp_over_rs = 0.12086
period = 3.5247486
sma_over_rs = 8.76
eccentricity = 0.0
inclination = 86.71     
periastron = 0.0
mid_time = 2452826.62928
albedo = 0.15 # assumed
emissivity = 1.0 # assumed

## 2. Core calculations

### 2.1 ```plc.exotethys``` 

*calculates the limb-darkening coefficients*

PyLightcurve includes a wrapper for the [```ExoTETHyS```](https://github.com/ucl-exoplanets/ExoTETHyS) package ([Morello et al. 2020](https://iopscience.iop.org/article/10.3847/1538-3881/ab63dc)), to allow for the easy calclation of the limb-darkening coefficients:

``` python
limb_darkening_coefficients =  plc.exotethys(stellar_logg, stellar_temperature, stellar_metallicity, filter_name, method='claret', stellar_model='Phoenix_2018', wlrange=None)
```

Returns a 4-element array. If the limb-darkening law used is not ```claret```, the extra elements will be zero.

**Example 1**:
``` python 
filter_name='COUSINS_R', method='claret', stellar_model='Phoenix_2018'
```

In [3]:
plc.exotethys(
    stellar_logg, stellar_temperature, stellar_metallicity,
    filter_name = 'COUSINS_R',
)

File already here...  teff06000_logg4.00_MH0.0.pickle
File already here...  teff06000_logg4.50_MH0.0.pickle
File already here...  teff06100_logg4.00_MH0.0.pickle
File already here...  teff06100_logg4.50_MH0.0.pickle


array([ 0.38056031,  0.15565698,  0.38746772, -0.23680038])

**Example 2**:
``` python 
filter_name='COUSINS_R', method='quad', stellar_model='Phoenix_2018'
```

In [4]:
plc.exotethys(
    stellar_logg, stellar_temperature, stellar_metallicity,
    filter_name = 'COUSINS_R',
    method = 'quad',
)

File already here...  teff06000_logg4.00_MH0.0.pickle
File already here...  teff06000_logg4.50_MH0.0.pickle
File already here...  teff06100_logg4.00_MH0.0.pickle
File already here...  teff06100_logg4.50_MH0.0.pickle


array([0.44521087, 0.16232214, 0.        , 0.        ])

**Example 3**:
``` python 
filter_name='COUSINS_R', method='linear', stellar_model='Phoenix_2018'
```

In [5]:
plc.exotethys(
    stellar_logg, stellar_temperature, stellar_metallicity,
    filter_name = 'COUSINS_R',
    method = 'linear',
)

File already here...  teff06000_logg4.00_MH0.0.pickle
File already here...  teff06000_logg4.50_MH0.0.pickle
File already here...  teff06100_logg4.00_MH0.0.pickle
File already here...  teff06100_logg4.50_MH0.0.pickle


array([0.53722501, 0.        , 0.        , 0.        ])

**Example 4**:
``` python 
filter_name='COUSINS_R', method='quad', stellar_model='Atlas_2000'
```

In [6]:
plc.exotethys(
    stellar_logg, stellar_temperature, stellar_metallicity,
    filter_name = 'COUSINS_R',
    stellar_model = 'Atlas_2000',
)

File already here...  teff06000_logg4.0_MH0.0.pickle
File already here...  teff06000_logg4.5_MH0.0.pickle
File already here...  teff06250_logg4.0_MH0.0.pickle
File already here...  teff06250_logg4.5_MH0.0.pickle


array([ 0.21057386,  1.07824683, -0.85391282,  0.2530806 ])

**Example 5**:
``` python 
filter_name='hst_wfc3_ir_g141', method='claret', stellar_model='Phoenix_2018', wlrange = [11000, 12000]
```

In [7]:
plc.exotethys(
    stellar_logg, stellar_temperature, stellar_metallicity,
    filter_name = 'hst_wfc3_ir_g141',
    wlrange = [11000, 12000],
)

File already here...  teff06000_logg4.00_MH0.0.pickle
File already here...  teff06000_logg4.50_MH0.0.pickle
File already here...  teff06100_logg4.00_MH0.0.pickle
File already here...  teff06100_logg4.50_MH0.0.pickle


array([ 0.34534734,  0.22695593, -0.00705248, -0.05772022])

**Example 6**:
``` python 
filter_name='hst_wfc3_ir_g141', method='claret', stellar_model='Phoenix_2018', wlrange = [12000, 13000]
```

In [ ]:
plc.exotethys(
    stellar_logg, stellar_temperature, stellar_metallicity,
    filter_name = 'hst_wfc3_ir_g141',
    wlrange = [12000, 13000],
)

File already here...  teff06000_logg4.00_MH0.0.pickle
File already here...  teff06000_logg4.50_MH0.0.pickle
File already here...  teff06100_logg4.00_MH0.0.pickle
File already here...  teff06100_logg4.50_MH0.0.pickle


**Example 7 (fails)**:
``` python 
filter_name='hst_wfc3_ir_g141', method='claret', stellar_model='Phoenix_2018', wlrange = [120000, 130000]
```

In [ ]:
# plc.exotethys(
#     stellar_logg, stellar_temperature, stellar_metallicity,
#     filter_name = 'hst_wfc3_ir_g141',
#     wlrange = [120000, 130000],
# )

### 2.2 ```plc.fp_over_fs``` 

*calculates the planet-to-star flux ratio (flux emmitted + reflected by the planet)*

Here, the ```filter_name``` and ```wlrange``` parameters can be used in the same way as in the ```plc.exotethys``` funcion.

In [ ]:
plc.fp_over_fs(
    rp_over_rs, sma_over_rs, albedo, emissivity, 
    stellar_temperature, filter_name='COUSINS_R')

### 2.3 ```plc.convert_to_bjd_tdb```

*converts a time array from different time formats to BJD<sub>TDB</sub>*

```time_format``` defines the format of the input time-series, and it can be: 
- ```JD_UTC```
- ```MJD_UTC```
- ```BJD_UTC```
- ```BJD_TDB```
- ```BJD_TT```
- ```HJD_UTC```
- ```HJD_BJD```
- ```HJD_TT```

In [ ]:
time_array_in_jd_utc = np.arange(2460000.0, 2460010.0, 0.1)

time_array_in_bjd_tdb = plc.convert_to_bjd_tdb(
    ra, dec, time_array_in_jd_utc, time_format='JD_UTC')

plt.figure(figsize=(7, 5))
plt.plot(
    time_array_in_bjd_tdb, 
    (time_array_in_bjd_tdb - time_array_in_jd_utc) * (24 * 60), 'ko')
plt.ylabel('$BJD_{TDB}$ - $JD_{UTC}$ (min)')
plt.xlabel('Time ($BJD_{TDB}$)')

### 2.4 ```plc.convert_to_jd_utc```

*converts a time array from different time formats to JD<sub>UTC</sub>*

```time_format``` defines the format of the input time-series, and it can be: 
- ```JD_UTC```
- ```MJD_UTC```
- ```BJD_UTC```
- ```BJD_TDB```
- ```BJD_TT```
- ```HJD_UTC```
- ```HJD_BJD```
- ```HJD_TT```

In [ ]:
time_array_in_bjd_tdb = np.arange(2460000.0, 2460010.0, 0.1)

time_array_in_jd_utc = plc.convert_to_jd_utc(
    ra, dec, time_array_in_bjd_tdb, time_format='BJD_TDB')

plt.figure(figsize=(7, 5))
plt.plot(
    time_array_in_bjd_tdb, 
    (time_array_in_bjd_tdb - time_array_in_jd_utc) * (24 * 60), 'ko')
plt.ylabel('$BJD_{TDB}$ - $JD_{UTC}$ (min)')
plt.xlabel('Time ($BJD_{TDB}$)')

### 2.5 ```plc.convert_to_relflux```

*converts a flux array from magnitude to relative flux*

```flux_format``` defines the format of the input series, and it can be: 
- ```flux```
- ```mag```

In [ ]:
plc.convert_to_relflux(np.array([9, 9.1, 9.2]), np.array([0.01, 0.01, 0.01]), 'mag')

### 2.6 ```plc.planet_orbit```

*calculates the (x,y,z) position of the planet in a cartesian frame with the star at (0,0,0), the Earth at (+infinity,0,0) and the periastron located on the −y axis for $\omega$=0.*

In [ ]:
time_array = np.arange(mid_time - 1.5*period, mid_time + 1.5*period, 0.01)

x, y, z = plc.planet_orbit(
    period, sma_over_rs, eccentricity, inclination, periastron, mid_time, time_array)


plt.figure()
plt.plot(time_array, x, 'ko', label='x')
plt.plot(time_array, y, 'ro', label='y')
plt.plot(time_array, z, 'bo', label='z')
plt.legend()
plt.ylabel('Planet-star distance ($R_*$)')
plt.xlabel('Time ($BJD_{TDB}$)')

### 2.7 ```plc.planet_star_projected_distance```

*calculates the distance between the planet and the star relatively to the stellar radius, as projected on the celestial sphere.*

In [ ]:
time_array = np.arange(mid_time - 1.5*period, mid_time + 1.5*period, 0.01)

planet_star_projected_distance = plc.planet_star_projected_distance(
    period, sma_over_rs, eccentricity, inclination, periastron, mid_time, time_array)


plt.figure()
plt.plot(time_array, planet_star_projected_distance, 'ko')
plt.ylabel('Planet-star projected distance ($R_*$)')
plt.xlabel('Time ($BJD_{TDB}$)')

### 2.8 ```plc.planet_phase```

*calculates the planet phase with respect to the transit mid-time (time of conjunction when the planet is in front of the star, no units)*

In [ ]:
time_array = np.arange(mid_time - 1.5*period, mid_time + 1.5*period, 0.01)

planet_phase = plc.planet_phase(period, mid_time, time_array)


plt.figure()
plt.plot(time_array, planet_phase, 'ko')
plt.ylabel('Planet phase')
plt.xlabel('Time ($BJD_{TDB}$)')

### 2.9 ```plc.transit_depth```

*calculates the transit depth (in relative flux)*

In [ ]:
limb_darkening_coefficients = plc.exotethys(
    stellar_logg, stellar_temperature, stellar_metallicity,
    filter_name = 'COUSINS_R'
)


plc.transit_depth(
    limb_darkening_coefficients, rp_over_rs, period, sma_over_rs, eccentricity, 
    inclination, periastron, method='claret', precision=2)

### 2.10 ```plc.transit_duration```

*calculates the transit duration (in days)*

In [ ]:
plc.transit_duration(
    rp_over_rs, period, sma_over_rs, eccentricity, inclination, periastron)

### 2.11 ```plc.transit_t12```

*calculates the time between transit contact points 1 and 2 (in days)*

In [ ]:
plc.transit_t12(rp_over_rs, period, sma_over_rs, eccentricity, inclination, periastron)

### 2.12 ```plc.eclipse_mid_time```

*calculates the eclipse mid-time (time of conjunction when the planet is behind the star, in days BJD<sub>TDB</sub>)*

In [ ]:
plc.eclipse_mid_time(period, sma_over_rs, eccentricity, inclination, periastron, mid_time)

### 2.13 ```plc.eclipse_depth```

*calculates the eclipse depth (in relative flux)*

In [ ]:
fp_over_fs = plc.fp_over_fs(rp_over_rs, sma_over_rs, albedo, 
                            emissivity, stellar_temperature, filter_name='COUSINS_R')

plc.eclipse_depth(fp_over_fs, rp_over_rs, period, sma_over_rs, 
                  eccentricity, inclination, periastron, precision=2)

### 2.14 ```plc.eclipse_duration```

*calculates the eclipse duration (in days)*

In [ ]:
plc.eclipse_duration(
    rp_over_rs, period, sma_over_rs, eccentricity, inclination, periastron)

### 2.15 ```plc.transit```

*calculates the transit model (in relative flux)*

In [ ]:
# limb_darkening_coefficients = plc.exotethys(
#     stellar_logg, stellar_temperature, stellar_metallicity,
#     filter_name = 'COUSINS_R'
# )

time_array = np.arange(mid_time - 0.1, mid_time + 0.1, 0.001)

flux = plc.transit(
    limb_darkening_coefficients, rp_over_rs, period, sma_over_rs, 
    eccentricity, inclination, periastron, mid_time, time_array, 
    method='claret', precision=2)

plt.figure()
plt.plot(time_array, flux, 'ko')
plt.ylabel('Flux (relative)')
plt.xlabel('Time ($BJD_{TDB}$)')


### 2.16 ```plc.eclipse```

*calculates the eclipse model (in relative flux)*

In [ ]:
fp_over_fs = plc.fp_over_fs(
    rp_over_rs, sma_over_rs, albedo, emissivity, 
    stellar_temperature, filter_name='COUSINS_R')

eclipse_mid_time = plc.eclipse_mid_time(
    period, sma_over_rs, eccentricity, inclination, periastron, mid_time)

time_array = np.arange(eclipse_mid_time - 0.1, eclipse_mid_time + 0.1, 0.001)

flux = plc.eclipse(
    fp_over_fs, rp_over_rs, period, sma_over_rs, eccentricity, 
    inclination, periastron, eclipse_mid_time, time_array, precision=2)


plt.figure()
plt.plot(time_array, flux, 'ko')
plt.ylabel('Flux (relative)')
plt.xlabel('Time ($BJD_{TDB}$)')

### 2.17 ```plc.transit_integrated```

*calculates the exposure-integtared transit model (in relative flux)*

In [ ]:
# limb_darkening_coefficients = plc.exotethys(
#     stellar_logg, stellar_temperature, stellar_metallicity,
#     filter_name = 'COUSINS_R'
# )

time_array = np.arange(mid_time - 0.1, mid_time + 0.1, 0.001)

flux = plc.transit_integrated(
    limb_darkening_coefficients, rp_over_rs, period, sma_over_rs, 
    eccentricity, inclination, periastron, mid_time, time_array, 
    exp_time=100, method='claret', precision=2)

plt.figure()
plt.plot(time_array, flux, 'ko')
plt.ylabel('Flux (relative)')
plt.xlabel('Time ($BJD_{TDB}$)')

### 2.18 ```plc.eclipse_integrated```

*calculates the exposure-integtared eclipse model (in relative flux)*

In [ ]:
fp_over_fs = plc.fp_over_fs(
    rp_over_rs, sma_over_rs, albedo, emissivity, 
    stellar_temperature, filter_name='COUSINS_R')

eclipse_mid_time = plc.eclipse_mid_time(
    period, sma_over_rs, eccentricity, inclination, periastron, mid_time)

time_array = np.arange(eclipse_mid_time - 0.1, eclipse_mid_time + 0.1, 0.001)

flux = plc.eclipse_integrated(
    fp_over_fs, rp_over_rs, period, sma_over_rs, eccentricity, 
    inclination, periastron, eclipse_mid_time, time_array, 
    exp_time=100, precision=2)


plt.figure()
plt.plot(time_array, flux, 'ko')
plt.ylabel('Flux (relative)')
plt.xlabel('Time ($BJD_{TDB}$)')